In [1]:
import pyspark
from pyspark.sql import SparkSession,types
import pandas as pd

In [2]:
spark = SparkSession.builder.master("local[*]").appName('rdd').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/04/17 07:04:54 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.25.167.220 instead (on interface eth0)
23/04/17 07:04:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/17 07:04:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [6]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-27 23:23:13|  2020-01-27 23:33:22|                 N|         1|         260|         223|              1|         2.98|       11.5|  0.5|    0.

In [7]:
df_green.rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 23, 23, 13), lpep_dropoff_datetime=datetime.datetime(2020, 1, 27, 23, 33, 22), store_and_fwd_flag='N', RatecodeID=1, PULocationID=260, DOLocationID=223, passenger_count=1, trip_distance=2.98, fare_amount=11.5, extra=0.5, mta_tax=0.5, tip_amount=3.2, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=16.0, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 20, 30), lpep_dropoff_datetime=datetime.datetime(2020, 1, 30, 20, 51), store_and_fwd_flag=None, RatecodeID=None, PULocationID=238, DOLocationID=47, passenger_count=None, trip_distance=7.65, fare_amount=25.46, extra=2.75, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=28.51, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 18, 13)

In [12]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [24]:
rdd = df_green.\
    select('lpep_pickup_datetime','PULocationID','total_amount','tip_amount').\
    rdd

In [25]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 23, 23, 13), PULocationID=260, total_amount=16.0, tip_amount=3.2),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 20, 30), PULocationID=238, total_amount=28.51, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 18, 13), PULocationID=82, total_amount=22.11, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 18, 12, 24), PULocationID=42, total_amount=8.8, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 11, 53), PULocationID=82, total_amount=19.02, tip_amount=0.0)]

#### sql where -> rdd filter

```
    works on the principle of True and False return,
    show rows which are true
```

In [26]:
from datetime import datetime
start = datetime(year=2020,month=1,day=1)

In [27]:
rdd.filter(lambda row:row.lpep_pickup_datetime>=start).take(10)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 23, 23, 13), PULocationID=260, total_amount=16.0, tip_amount=3.2),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 20, 30), PULocationID=238, total_amount=28.51, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 18, 13), PULocationID=82, total_amount=22.11, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 18, 12, 24), PULocationID=42, total_amount=8.8, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 11, 53), PULocationID=82, total_amount=19.02, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 15, 35), PULocationID=7, total_amount=47.42, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 2, 11, 32), PULocationID=66, total_amount=12.3, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 9, 40, 17), PULocationID=42, total_amount=7.3, tip_amount=0.0),
 Row(lpep_pickup_datetime=datetime.da

### sql group by -> map+reduce

In [32]:
def mapper(row):
    hour = row.lpep_pickup_datetime.replace(minute=0,second=0,microsecond=0)
    zone = row.PULocationID

    amount = row.total_amount
    tip = row.tip_amount
    count = 1

    key = (hour, zone)
    value = (amount,tip,count)

    return (key, value)

In [33]:
rdd.\
    filter(lambda row:row.lpep_pickup_datetime>=start).\
    map(mapper).take(5)

[((datetime.datetime(2020, 1, 27, 23, 0), 260), (16.0, 3.2, 1)),
 ((datetime.datetime(2020, 1, 30, 20, 0), 238), (28.51, 0.0, 1)),
 ((datetime.datetime(2020, 1, 22, 18, 0), 82), (22.11, 0.0, 1)),
 ((datetime.datetime(2020, 1, 4, 18, 0), 42), (8.8, 0.0, 1)),
 ((datetime.datetime(2020, 1, 27, 11, 0), 82), (19.02, 0.0, 1))]

In [55]:
def reducer(val1,val2):
    val1_amount,val1_tip,val1_count = val1
    val2_amount,val2_tip,val2_count = val2

    output_amount = val1_amount+val2_amount
    output_tip = (val1_tip*val1_count+val2_tip*val2_count)/(val1_count+val2_count)
    output_count = val1_count+val2_count

    return (output_amount,output_tip,output_count)


In [56]:
rdd.\
    filter(lambda row:row.lpep_pickup_datetime>=start).\
    map(mapper).\
    reduceByKey(reducer).take(5)

[((datetime.datetime(2020, 1, 27, 23, 0), 260),
  (154.26, 0.7506666666666665, 15)),
 ((datetime.datetime(2020, 1, 4, 9, 0), 42),
  (426.55000000000024, 0.18777777777777777, 36)),
 ((datetime.datetime(2020, 1, 11, 18, 0), 41),
  (476.0100000000002, 0.7775555555555554, 45)),
 ((datetime.datetime(2020, 1, 22, 12, 0), 244),
  (352.26000000000016, 0.7647619047619049, 21)),
 ((datetime.datetime(2020, 1, 22, 15, 0), 177), (56.28, 0.0, 3))]

In [57]:
# converting into unnested structure
def unwrap(row):
    return (row[0][0],row[0][1],row[1][0],row[1][1],row[1][2])

In [59]:
rdd.\
    filter(lambda row:row.lpep_pickup_datetime>=start).\
    map(mapper).\
    reduceByKey(reducer).\
    map(unwrap).take(5)

[(datetime.datetime(2020, 1, 27, 23, 0), 260, 154.26, 0.7506666666666665, 15),
 (datetime.datetime(2020, 1, 4, 9, 0),
  42,
  426.55000000000024,
  0.18777777777777777,
  36),
 (datetime.datetime(2020, 1, 11, 18, 0),
  41,
  476.0100000000002,
  0.7775555555555554,
  45),
 (datetime.datetime(2020, 1, 22, 12, 0),
  244,
  352.26000000000016,
  0.7647619047619049,
  21),
 (datetime.datetime(2020, 1, 22, 15, 0), 177, 56.28, 0.0, 3)]

In [60]:
# converting to df -> schema will be lost
rdd.\
    filter(lambda row:row.lpep_pickup_datetime>=start).\
    map(mapper).\
    reduceByKey(reducer).\
    map(unwrap).\
    toDF().\
    show()

+-------------------+---+------------------+-------------------+---+
|                 _1| _2|                _3|                 _4| _5|
+-------------------+---+------------------+-------------------+---+
|2020-01-27 23:00:00|260|            154.26| 0.7506666666666665| 15|
|2020-01-04 09:00:00| 42|426.55000000000024|0.18777777777777777| 36|
|2020-01-11 18:00:00| 41| 476.0100000000002| 0.7775555555555554| 45|
|2020-01-22 12:00:00|244|352.26000000000016| 0.7647619047619049| 21|
|2020-01-22 15:00:00|177|             56.28|                0.0|  3|
|2020-01-17 14:00:00|  7| 350.2100000000001| 0.6464705882352941| 34|
|2020-01-15 21:00:00| 74|458.28000000000014| 1.0058333333333336| 36|
|2020-01-19 12:00:00|166| 775.6199999999997|  1.741702127659575| 47|
|2020-01-29 08:00:00| 75| 767.3899999999995| 1.1294915254237288| 59|
|2020-01-19 00:00:00| 25|            142.75|               0.95| 11|
|2020-01-28 10:00:00| 41| 523.9100000000001|  1.151621621621622| 37|
|2020-01-03 15:00:00|134|126.17999

In [62]:
# for schema using namedtuple
from collections import namedtuple
RevenueRow = namedtuple('RevenueRow',['hour','zone','revenue','tip','count'])

In [63]:
# converting into unnested structure
def unwrapNamed(row):
    return RevenueRow(row[0][0],row[0][1],row[1][0],row[1][1],row[1][2])

In [ ]:
from pyspark.sql import 

In [67]:
# converting to df -> schema will be there
df_rdd_final = rdd.\
    filter(lambda row:row.lpep_pickup_datetime>=start).\
    map(mapper).\
    reduceByKey(reducer).\
    map(unwrapNamed).\
    toDF()

In [69]:
df_rdd_final.show()

+-------------------+----+------------------+-------------------+-----+
|               hour|zone|           revenue|                tip|count|
+-------------------+----+------------------+-------------------+-----+
|2020-01-27 23:00:00| 260|            154.26| 0.7506666666666665|   15|
|2020-01-04 09:00:00|  42|426.55000000000024|0.18777777777777777|   36|
|2020-01-11 18:00:00|  41| 476.0100000000002| 0.7775555555555554|   45|
|2020-01-22 12:00:00| 244|352.26000000000016| 0.7647619047619049|   21|
|2020-01-22 15:00:00| 177|             56.28|                0.0|    3|
|2020-01-17 14:00:00|   7| 350.2100000000001| 0.6464705882352941|   34|
|2020-01-15 21:00:00|  74|458.28000000000014| 1.0058333333333336|   36|
|2020-01-19 12:00:00| 166| 775.6199999999997|  1.741702127659575|   47|
|2020-01-29 08:00:00|  75| 767.3899999999995| 1.1294915254237288|   59|
|2020-01-19 00:00:00|  25|            142.75|               0.95|   11|
|2020-01-28 10:00:00|  41| 523.9100000000001|  1.151621621621622

In [72]:
schema = types.StructType([
            types.StructField('hour',types.TimestampType(), True), 
            types.StructField('zone',types.LongType(), True), 
            types.StructField('revenue',types.DoubleType(), True), 
            types.StructField('tip',types.DoubleType(), True), 
            types.StructField('count',types.LongType(), True)])

In [73]:
# converting to df -> schema will be there
df_rdd_final = rdd.\
    filter(lambda row:row.lpep_pickup_datetime>=start).\
    map(mapper).\
    reduceByKey(reducer).\
    map(unwrapNamed).\
    toDF(schema)

In [74]:
df_rdd_final.show()

+-------------------+----+------------------+-------------------+-----+
|               hour|zone|           revenue|                tip|count|
+-------------------+----+------------------+-------------------+-----+
|2020-01-27 23:00:00| 260|            154.26| 0.7506666666666665|   15|
|2020-01-04 09:00:00|  42|426.55000000000024|0.18777777777777777|   36|
|2020-01-11 18:00:00|  41| 476.0100000000002| 0.7775555555555554|   45|
|2020-01-22 12:00:00| 244|352.26000000000016| 0.7647619047619049|   21|
|2020-01-22 15:00:00| 177|             56.28|                0.0|    3|
|2020-01-17 14:00:00|   7| 350.2100000000001| 0.6464705882352941|   34|
|2020-01-15 21:00:00|  74|458.28000000000014| 1.0058333333333336|   36|
|2020-01-19 12:00:00| 166| 775.6199999999997|  1.741702127659575|   47|
|2020-01-29 08:00:00|  75| 767.3899999999995| 1.1294915254237288|   59|
|2020-01-19 00:00:00|  25|            142.75|               0.95|   11|
|2020-01-28 10:00:00|  41| 523.9100000000001|  1.151621621621622

In [75]:
df_rdd_final.write.parquet('data/rdd/report/revenue/ex1')

23/04/16 10:45:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### mapPartitions

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [5]:
columns = ['VendorID','lpep_pickup_datetime','PULocationID','DOLocationID','trip_distance']
duration_rdd = df_green.select(columns).rdd

In [7]:
duration_rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 23, 23, 13), PULocationID=260, DOLocationID=223, trip_distance=2.98),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 30, 20, 30), PULocationID=238, DOLocationID=47, trip_distance=7.65),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 18, 13), PULocationID=82, DOLocationID=95, trip_distance=1.83),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 18, 12, 24), PULocationID=42, DOLocationID=244, trip_distance=1.65),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 11, 53), PULocationID=82, DOLocationID=197, trip_distance=6.38)]

In [19]:
def apply_model_to_each_partition(partition):
    cnt = 0
    for i in partition:
        cnt+=1
    return [cnt]

In [20]:
duration_rdd.mapPartitions(apply_model_to_each_partition).collect()

[902254, 371456, 281352, 275214, 275248, 267149, 225041, 125258]

In [21]:
sum(duration_rdd.mapPartitions(apply_model_to_each_partition).collect())

2722972

In [23]:
df_green.count()

2722972

#### trying to parition into equals, just for learning

In [17]:
duration_rdd_equals = duration_rdd.zipWithIndex().map(lambda x: (x[1],x[0])).partitionBy(8,lambda p:p).values()

In [32]:
duration_rdd_equals.mapPartitions(apply_model_to_each_partition).collect()

[340372, 340372, 340372, 340372, 340371, 340371, 340371, 340371]

In [35]:
def apply_model_to_each_partition(partition):
    df = pd.DataFrame(partition,columns=columns)
    cnt = len(df)
    return [cnt]

In [36]:
duration_rdd_equals.mapPartitions(apply_model_to_each_partition).collect()

[340372, 340372, 340372, 340372, 340371, 340371, 340371, 340371]

#### predicting model

In [18]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

In [19]:
def apply_model_to_each_partition(partition):
    df = pd.DataFrame(partition,columns=columns)
    df['duration_pred'] = model_predict(df)
    for row in df.itertuples():
        yield row

In [20]:
predicted_rdd = duration_rdd_equals.mapPartitions(apply_model_to_each_partition)

In [157]:
predicted_rdd.take(5)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-27 23:23:13'), PULocationID=260, DOLocationID=223, trip_distance=2.98, duration_pred=14.9),
 Pandas(Index=1, VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-18 21:41:43'), PULocationID=129, DOLocationID=129, trip_distance=0.4, duration_pred=2.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-04 18:02:05'), PULocationID=55, DOLocationID=26, trip_distance=4.99, duration_pred=24.950000000000003),
 Pandas(Index=3, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-19 14:13:00'), PULocationID=213, DOLocationID=74, trip_distance=5.26, duration_pred=26.299999999999997),
 Pandas(Index=4, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-13 15:58:00'), PULocationID=121, DOLocationID=258, trip_distance=5.52, duration_pred=27.599999999999998)]

In [158]:
schema=types.StructType([
    types.StructField('Index', types.LongType(), True), 
    types.StructField('VendorID', types.DoubleType(), True), 
    types.StructField('lpep_pickup_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('trip_distance', types.DoubleType(), True), 
    types.StructField('duration_pred', types.DoubleType(), True)])

In [182]:
from datetime import datetime
def change_data_type(row):
    #return (row[0],row[1],datetime.strptime(str(row[2]), "%Y-%m-%d %H:%M:%S"),row[3],row[4],row[5],row[6])
    return (row[0],row[1],row[2].to_pydatetime(),row[3],row[4],row[5],row[6])

In [183]:
predicted_rdd_ = predicted_rdd.map(change_data_type)

In [184]:
predicted_rdd_.take(5)

[(0, 2.0, datetime.datetime(2020, 1, 27, 23, 23, 13), 260, 223, 2.98, 14.9),
 (1, 1.0, datetime.datetime(2020, 1, 18, 21, 41, 43), 129, 129, 0.4, 2.0),
 (2,
  2.0,
  datetime.datetime(2020, 1, 4, 18, 2, 5),
  55,
  26,
  4.99,
  24.950000000000003),
 (3,
  nan,
  datetime.datetime(2020, 1, 19, 14, 13),
  213,
  74,
  5.26,
  26.299999999999997),
 (4,
  nan,
  datetime.datetime(2020, 1, 13, 15, 58),
  121,
  258,
  5.52,
  27.599999999999998)]

In [190]:
final_df = predicted_rdd_.toDF(schema).drop('Index')

In [191]:
final_df.show()

+--------+--------------------+------------+------------+-------------+------------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|     duration_pred|
+--------+--------------------+------------+------------+-------------+------------------+
|     2.0| 2020-01-27 23:23:13|         260|         223|         2.98|              14.9|
|     1.0| 2020-01-18 21:41:43|         129|         129|          0.4|               2.0|
|     2.0| 2020-01-04 18:02:05|          55|          26|         4.99|24.950000000000003|
|     NaN| 2020-01-19 14:13:00|         213|          74|         5.26|26.299999999999997|
|     NaN| 2020-01-13 15:58:00|         121|         258|         5.52|27.599999999999998|
|     1.0| 2020-01-22 16:42:40|          49|         189|          0.0|               0.0|
|     NaN| 2020-01-28 18:39:00|          74|          74|         23.1|             115.5|
|     2.0| 2020-01-15 08:49:59|          41|          74|         0.86|               4.3|

In [192]:
final_df.printSchema()

root
 |-- VendorID: double (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- duration_pred: double (nullable = true)

